# Análise Titanic

![Imagem](Files/img.png)

 - Link para [primeira analise](https://github.com/DevTheo25/Titanic_ML/blob/main/Primeira_analise.ipynb). 

## Segunda Análise

 - Nessa análise vamos utilizar mais colunas do Data Set, fazer os tratamentos necessários dessas colunas, entender e tratar colunas de texto.

In [50]:
# Importando pandas
import pandas as pd

In [51]:
# Lendo e visualizando a base de dados Treino
train = pd.read_csv("Files/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
# Lendo e visualizando a base de dados Teste
test = pd.read_csv("Files/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [53]:
# Vizualisando a cardinalidade dos dados
train.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Pclass           3
Embarked         3
Survived         2
Sex              2
dtype: int64

In [54]:
# Eliminando colunas para os dados de treino
train = train.drop(["Name", "Ticket", "Cabin"], axis=1)

In [55]:
# Elimnando colunas para os dados de teste
test = test.drop(["Name", "Ticket", "Cabin"], axis=1)

### Tratando colunas com valores nulos

In [56]:
# Caculando a média da coluna Age
train["Age"].mean()

29.69911764705882

In [57]:
# Alterando os valores nulos na coluna Age, pela média base de treino
train.loc[train.Age.isnull(), "Age"] = train["Age"].mean()

In [58]:
# Alterando os valores nulos na coluna Age, pela média base de teste
test.loc[test.Age.isnull(), "Age"] = test["Age"].mean()

In [59]:
train.isnull().sum().sort_values(ascending=False)

Embarked       2
PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

In [60]:
test.isnull().sum().sort_values(ascending=False)

Fare           1
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Embarked       0
dtype: int64

In [61]:
# Alterando valores nulos da coluna Fare
test.loc[test.Fare.isnull(), "Fare"] = test.Fare.mean()

In [62]:
test.isnull().sum().sort_values(ascending=False)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

----

 # Entendendo colunas de texto

- Vamos agora fazer o tratamento das colunas de texto

In [63]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


- Temos 2 colunas de texto, **Sex** e **Embarked**

In [64]:
train["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

- Para coluna **Sex** podemos converter para binário por existir apenas 2 valores diferentes

In [65]:
# Convertendo a coluna Sex para binário
train['Sex_binary'] = train["Sex"].apply(lambda x: 1 if  x == "female" else 0)

In [66]:
# Fazendo a mesma conversão para base de teste
test['Sex_binary'] = test["Sex"].apply(lambda x: 1 if  x == "female" else 0)

In [67]:
# Eliminando a antiga coluna Sex das bases
train = train.drop(["Sex"], axis=1)
test = test.drop("Sex", axis=1)

- Para a coluna **Embarked** precisanmos utiliza **one-hot encoding**. Isso significa que cada valor distinto na coluna "Embarked" será transformado em uma nova coluna binária

In [68]:
train["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [69]:
# Aplicar one-hot encoding
encoded = pd.get_dummies(train['Embarked'], prefix='Embarked')

# Concatenar os novos dados codificados com o dataframe original
train = pd.concat([train, encoded], axis=1)

# Elimindo coluna "Embarked"
train = train.drop("Embarked", axis=1)
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_binary,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,0,0,1


In [70]:
# Aplicando o mesmo conceito para a base de Teste
encoded = pd.get_dummies(test['Embarked'], prefix='Embarked')

# Concatenar os novos dados codificados com o dataframe original
test = pd.concat([test, encoded], axis=1)

# Elimindo coluna "Embarked"
test = test.drop("Embarked", axis=1)
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_binary,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,1
2,894,2,62.0,0,0,9.6875,0,0,1,0
3,895,3,27.0,0,0,8.6625,0,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,1


# Criando Modelos

- Separando a base de treino entre Treino e Validação

In [71]:
# Serpando X e y
X = train.drop(["Survived", "PassengerId"], axis=1)
y = train["Survived"]

In [72]:
# Importando train_test_split
from sklearn.model_selection import train_test_split

In [73]:
treino_X, valid_X, treino_y, valid_y = train_test_split(X, y, random_state=1)

- Modelo Random Forent

In [74]:
# Importando modelo e acurácia
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [75]:
# Treinando Random Forest
modelo_rf = RandomForestClassifier(random_state=1)
modelo_rf.fit(treino_X, treino_y)

RandomForestClassifier(random_state=1)

In [76]:
# Predizendo os dados de validação
predicao_rf = modelo_rf.predict(valid_X)

In [77]:
# Acurácia do Modelo Random Forest
acuracia_rf = accuracy_score(predicao_rf, valid_y)

- Modelo KNeighborsClassifer

In [78]:
# Importando modelo
from sklearn.neighbors import KNeighborsClassifier

In [79]:
# Treinando o Modelo
modelo_kn = KNeighborsClassifier(n_neighbors=3)
modelo_kn.fit(treino_X, treino_y)

KNeighborsClassifier(n_neighbors=3)

In [80]:
## Predizendo os dados de validação
predicao_kn = modelo_kn.predict(valid_X)

In [81]:
# Acurácia
acuracia_kn = accuracy_score(valid_y, predicao_kn)

- Modelo Regressão Logística

In [82]:
# Importando modelo
from sklearn.linear_model import LogisticRegression

In [83]:
# Treinando o Modelo
modelo_lr = LogisticRegression(random_state=1, max_iter=1000)
modelo_lr.fit(treino_X, treino_y)

LogisticRegression(max_iter=1000, random_state=1)

In [84]:
# Predizendo os dados de Validação
predicao_lr = modelo_lr.predict(valid_X)

In [85]:
# Acurácia
acuracia_lr = accuracy_score(valid_y, predicao_lr)

## Resultados dos Modelos

In [86]:
print(f"Acurácia Modelo RandomForest: {acuracia_rf}\n")
print(f"Acurácia Modelo KNeighborsClassifer: {acuracia_kn}\n")
print(f"Acurácia Modelo LogisticRegression: {acuracia_lr}")

Acurácia Modelo RandomForest: 0.7757847533632287

Acurácia Modelo KNeighborsClassifer: 0.7309417040358744

Acurácia Modelo LogisticRegression: 0.7937219730941704


---

#  Predizendo os dados de Teste com o modelo Regressão Logística

In [92]:
X_train = train.drop(["PassengerId", "Survived"], axis=1)
y_train = train["Survived"]

In [93]:
X_test = test.drop("PassengerId", axis=1)

In [94]:
# Treinando o modelo Regressão Logística
modelo_final = LogisticRegression(random_state=1, max_iter=1000)
modelo_final.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [95]:
# Predizendo os dados de Validação
predicao_final = modelo_final.predict(X_test)

In [96]:
sub = pd.Series(predicao_final, index=test["PassengerId"], name="Survived")
sub.to_csv("Files/Segunda_predicao.csv", header=True)